# Data Preprocessing for Metal Nanoparticle Surface Atoms Clustering

This jupyter notebook is used to preprocess the collected tabular data from metal nanoparticle data sets.

## Index:
- [Requirements](#requirements)
- [Import Libraries](#import-libraries)
- [Load Data](#load-data)
- [Data Preprocessing](#data-preprocessing)
    - [Description & Massage](#desc-massage)
    - [Data Scaling](#data-scaling)
    - [Feature Selection](#feat-select)
    - [Data Visualisation](#data-vis)
- [Test Cases](#test-cases)
- [Variables Storage](#var-store)

<a id='requirements'></a>
## Requirements

The following files should be available in a specified directory (./data/ here):
* "dfAlls.pickle"

<a id='import-libraries'></a>
## Import Libraries

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import sys
import warnings

import math
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib import rcParams
import numpy as np
# import modin.pandas as modpd
import pandas as pd
import pickle
import seaborn as sns
import sklearn
from statsmodels.graphics.gofplots import qqplot

# Dimensionality Reduction
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# Machine Learning Miscellaneous
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, RobustScaler, StandardScaler, PowerTransformer, QuantileTransformer
from scipy.stats import probplot, shapiro, normaltest, anderson, skew, kurtosis

In [4]:
# Set up notebook environment
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# Settings for figures
sns.set_theme(context='paper', style='ticks', palette='colorblind', font='sans-serif', font_scale=1, color_codes=True, rc=None)  # Options are: {paper, notebook, talk, poster}
DPI = None
BINARY_COLOURS = ('#FE0101', '#0729F9')  # Red & blue from blue-white-red colour bar
SMALL_SIZE, MEDIUM_SIZE, LARGE_SIZE, TITLE_SIZE = 8, 10, 12, 14
plt.rc('font', size=LARGE_SIZE)  # controls default text sizes
plt.rc('axes', titlesize=TITLE_SIZE)  # fontsize of the axes title
plt.rc('axes', labelsize=LARGE_SIZE)  # fontsize of the x and y labels
plt.rc('xtick', labelsize=LARGE_SIZE)  # fontsize of the tick labels
plt.rc('ytick', labelsize=LARGE_SIZE)  # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)  # legend fontsize
plt.rc('figure', titlesize=TITLE_SIZE)  # fontsize of the figure title
# rcParams['axes.spines.top'] = False
# rcParams['axes.spines.right'] = False
# rcParams['figure.figsize'] = [8, 5]
# rcParams['figure.autolayout'] = True
# rcParams['font.style'] = 'normal'
# rcParams['xtick.labelsize'] = 10
# rcParams['ytick.labelsize'] = 10

warnings.filterwarnings("ignore")
%pprint

Pretty printing has been turned OFF


In [5]:
# Print version for reproducibility
print(f"matplotlib version {mpl.__version__}")
print(f"numpy version {np.__version__}")
print(f"pandas version {pd.__version__}")
print(f"seaborn version {sns.__version__}")
print(f"sklearn version {sklearn.__version__}")

matplotlib version 3.5.1
numpy version 1.21.5
pandas version 1.3.5
seaborn version 0.11.2
sklearn version 1.0.2


<a id='load-data'></a>
## Load Data

In [6]:
%store -r testCases
RANDOM_SEED = 42
NUM_JOBS = 8
CORR_METHOD = 'pearson'
PCA_EXP_VAR_THRESHOLD = 0.99

varThreshs, corrThreshs = [0.0], [0.9]
reinitialise = False
if reinitialise:
    dfScaledNoLVHCs = np.array([[[None] * len(testCases)] * len(corrThreshs)] * len(varThreshs))
    dfNoLVHCs = np.array([[[None] * len(testCases)] * len(corrThreshs)] * len(varThreshs))
    corrMatNoLVHCs = np.array([[[None] * len(testCases)] * len(corrThreshs)] * len(varThreshs)) 
    %store -d dfScaledNoLVHCs dfNoLVHCs corrMatNoLVHCs
    %store dfScaledNoLVHCs dfNoLVHCs corrMatNoLVHCs

with open('data/dfAlls.pickle', 'rb') as f: 
    dfAlls = pickle.load(f)
# for (i, testCase) in enumerate(testCases):
#     print("{0} {1}".format(i, testCase))
#     display(dfAlls[i].sample(5))

<a id='data-preprocessing'></a>
## Data Preprocessing

<a id='desc-massage'></a>
### Description & Massage

In [7]:
def dfDesc(df, interactive=True):
    """
    Describe input DataFrame using df.describe() and df.info()
    input:
        df = input DataFrame
        interactive = Boolean indicator to decide usage of function (display/print) for DataFrame inspection
    """
    print("\nOriginal DataFrame:")
    display(df.sample(5)) if interactive else print(df.sample(5))
    print("\nDataFrame description:")
    display(df.describe()) if interactive else print(df.describe())
    print("\nDataFrame information:")
    df.info()

In [8]:
def dfMassage(df, verbose=False, interactive=True):
    """
    Handle missing values, drop duplicates in input DataFrame, one-hot encode certain features, and turn columns datatype into numeric
    input:
        df = input DataFrame
        verbose = Boolean indicator for output printing
        interactive = Boolean indicator to decide usage of function (display/print) for DataFrame inspection
    output:
        dfNew = processed DataFrame
    """
    # Handle missing values
    if df.isna().any().any():
        print("Missing entries exist!")
        missingNum = df.isna().sum()
        missingNum[missingNum > 0].sort_values(ascending=False)
        print(f"Missing rows in each column/feature:\n{missingNum}")
        # df.dropna(axis=0, how='any', thresh=None, subset=['pg', 'csm', 'molRMS'], inplace=True)

    # Drop duplicates
    if df.duplicated().any():
        print("Duplicate entries exists!")
        print(f"Number of rows before dropping duplicates: {len(df)}")
        df.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=False);
        print(f"Number of rows after dropping duplicates: {len(dfNew)}")
        
    # One-hot encode features
    dummyEle = pd.get_dummies(df['ele'], prefix='ele')
    dfNew = pd.merge(left=df, right=dummyEle, left_index=True, right_index=True)
    
    # To numeric
    dfNew = dfNew.astype({'xNorm': float, 'yNorm': float, 'zNorm': float, 
                          'Ixx': float, 'Iyy': float, 'Izz': float, 
                          'degenDeg': float})
    
    print("\nMissing values handled, duplicates dropped, one-hot encoded categorical features, feature columns numericised:")
    if verbose: display(dfNew.sample(5)) if interactive else print(dfNew.sample(5))
    return dfNew

In [9]:
def dfDropUnusedCols(df):
    """
    Drop unused feature columns.
    input:
        df = input DataFrame
    output:
        dfNew = processed DataFrame
    """
    # 'x', 'y', 'z' are redundant (included normalised columns)
    # 'pg' could only be one-hot encoded when all possibilities are recorded
    # 'csm', 'molRMS' undefined when 'pg' is missing
    # inclusion of "E", "C8", etc. makes clustering difficult (binary features become dominant)
    # angParam, 
    pgIdx = list(df.columns).index('pg')
    surfIdx = list(df.columns).index('surf')
    asymFeats = ['blmin', 'angParam', 'chi1', 
                 'disord2', 'disord4', 'disord6', 'disord8', 'disord10', 'disord12', 
                 'disordAvg2', 'disordAvg4', 'disordAvg6', 'disordAvg8', 'disordAvg10', 'disordAvg12',
                 'Ixx', 'Iyy', 'Izz', 'csm', 'molRMS']
    cols2drop = ['x', 'y', 'z', 'ele'] + asymFeats + list(df.columns)[pgIdx:surfIdx] 
    dfNew = df.drop(labels=cols2drop, axis=1, index=None, columns=None, level=None, inplace=False, errors="raise")
    dfNew.rename(columns={'xNorm': 'x', 'yNorm': 'y', 'zNorm': 'z'}, inplace=True)
    return dfNew

<a id='data-scaling'></a>
### Data-Scaling

In [10]:
def dfScale(df, catCols=None, scaler="minMax"):
    """
    Scale input feature DataFrame using various sklearn scalers
    input:
        df = DataFrame with columns being features
        catCols = list of categorical features, not to be scaled
        scaler = type of sklearn scaler to use
    output:
        Xscaled = scaled DataFrames
    """
    X4keep = df[catCols] if catCols else None
    X4scale = df.drop(X4keep.columns, axis=1, inplace=False) if catCols else df
    if scaler == "minMax":
        scaler = MinMaxScaler(feature_range=(0, 1), copy=True).fit(X4scale)
    elif scaler == "stand":
        scaler = StandardScaler(copy=True, with_mean=True, with_std=True).fit(X4stand)
    elif scaler == "robust":
        scaler = RobustScaler(with_centering=True, with_scaling=True, quantile_range=(25.0, 75.0), copy=True).fit(X4scale)
    else:
        raise("\nScaler specified unknown!")
    arrXscaled = scaler.transform(X4scale)
    Xscaled = pd.concat(objs=[pd.DataFrame(arrXscaled, index=df.index, columns=X4scale.columns), X4keep], axis=1)
    Xscaled.index.name = "Index"
    return Xscaled

<a id='feat-select'></a>
### Feature Selection

In [11]:
def getLowVarCols(df, skipCols=None, varThresh=0.0, autoRemove=False, verbose=False):
    """
    Wrapper for sklearn VarianceThreshold.
    input:
        df = DataFrame with columns being features
        skipCols = columns to be skipped
        varThresh = low variance threshold for features to be detected
        autoRemove = Boolean indicator for automatic removal of low variance columns
        verbose = Boolean indicator for output printing
    output:
        df = DataFrame with low variance features removed
        lowVarCols = list of low variance features
    """
    print(f"\n  Finding features with low-variance, threshold: {varThresh}")
    try:
        allCols = df.columns
        if skipCols:
            remainCols = allCols.drop(skipCols)
            maxIdx = len(remainCols) - 1
            skippedIdx = [allCols.get_loc(col) for col in skipCols]

            # Adjust insert location by the number of columns removed (for non-zero insertion locations) to keep relative locations intact
            for idx, item in enumerate(skippedIdx):
                if item > maxIdx:
                    diff = item - maxIdx
                    skippedIdx[idx] -= diff
                if item == maxIdx:
                    diff = item - len(skipCols)
                    skippedIdx[idx] -= diff
                if idx == 0:
                    skippedIdx[idx] = item
            skippedVals = df.iloc[:, skippedIdx].values
        else:
            remainCols = allCols

        X = df.loc[:, remainCols].values
        vt = VarianceThreshold(threshold=varThresh)
        vt.fit(X)
        keepColsIdxs = vt.get_support(indices=True)
        keepCols = [remainCols[idx] for idx, _ in enumerate(remainCols) if idx in keepColsIdxs]
        lowVarCols = list(np.setdiff1d(remainCols, keepCols))
        if verbose: print("    Found {0} low-variance columns.".format(len(lowVarCols)))

        if autoRemove:
            if verbose: print("    Removing low-variance features...")
            X_removed = vt.transform(X)
            if verbose: print("    Reassembling the dataframe (with low-variance features removed)...")
            df = pd.DataFrame(data=X_removed, columns=keepCols)
            if skipCols:
                for (i, index) in enumerate(skippedIdx): df.insert(loc=index, column=skipCols[i], value=skippedVals[:, i])
            if verbose: print("    Succesfully removed low-variance columns: {0}.".format(lowVarCols))
        else:
            if verbose: print("    No changes have been made to the dataframe.")
    except Exception as e:
        print(e)
        print("    Could not remove low-variance features. Something went wrong.")
    return df, lowVarCols

In [12]:
def getHighCorCols(df, corrThresh=0.95, method="pearson", verbose=False):
    """
    Compute correlation matrix using pandas
    input: 
        df = input DataFrame with columns being features
        corrThresh = threshold to identify highly correlated features
        method = method to compute DataFrame correlation
        verbose = Boolean indicator for output printing
    output: 
        corrMat = correlation matrix of all features in input DataFrame
        highCorCols = tuples of highly correlated features
    """ 
    print(f"\n  Finding features highly-correlated with each other, threshold: {corrThresh}")
    corrMat = df.corr(method=method, min_periods=1)
    corrMatUpper = corrMat.where(np.triu(np.ones(corrMat.shape), k=1).astype(np.bool))  # Select upper triangle of correlation matrix
    highCorCols = [(row, col) for col in corrMatUpper.columns for row in corrMatUpper.index if corrMatUpper.loc[row, col] > corrThresh]
    if verbose: print(f"    Highly correlated columns: {highCorCols}")
    return corrMat, highCorCols

In [13]:
def plotCorrMat(corrMat, figSize=(8, 8), figName=None):
    """
    Wrapper for sklearn VarianceThreshold.
    input:
        corrMat = correlation matrix of all features in input DataFrame
        figSize = size of figure
        figName = path to save figure
    """
    sns.set(font_scale=1.1)
    # cmap = sns.diverging_palette(h_neg=210, h_pos=350, s=90, l=50, as_cmap=True)
    g = sns.clustermap(data=corrMat.abs().mul(100).astype(float), cmap='Blues', metric='correlation', figsize=figSize);
    plt.setp(g.ax_heatmap.yaxis.get_majorticklabels(), rotation=0);
    # x0, _y0, _w, _h = g.cbar_pos
    # print(g.cbar_pos)
    g.ax_cbar.set_position([0.04, 0.82, 0.05, 0.14]);
    if figName: 
        plt.savefig(figName, dpi=DPI, bbox_inches='tight');
    plt.close();

In [14]:
def autoDrop(highCorCols, verbose=False):
    """
    Automate the selection of highly-correlated features to be dropped. 
    Rank given to each feature is based on its degree of utility/ease of interpretation if it turns out to be correlated with target labels
    'x', 'y', 'z' not very useful even if found to be important
    'avg' values of bond geometries are more useful than 'num' values
    'max', 'min' of bond geometries are hard to control experimentally
    'angParam', 'centParam' are easier to interpret than 'entroParam'
    'disorder' parameters are easier to interpret than 'q' Steinhardt's parameters
    averaged 'q' parameters are more robust to thermal fluctuations, hence more useful than pure 'q' parameters
    q6 > q4 > q2 == q8 == q10 == q12 in usefulness based on literature, thus 'disorder' parameters follow same sequence
    input:
        highCorCols = list of tuples of highly-correlated features
        verbose = Boolean indicator for output printing
    output: 
        cols2drop = list of features to be dropped
    """
    utilityRanking = {'x': 10, 'y': 10, 'z': 10, 'rad': 0, 
                     'blavg': 2, 'blmax': 8, 'blmin': 8, 'blnum': 3, 
                     'ba1avg': 2, 'ba1max': 8, 'ba1min': 8, 'ba1num': 3,
                     'ba2avg': 2, 'ba2max': 8, 'ba2min': 8, 'ba2num': 3,
                     'btposavg': 2, 'btposmax': 8, 'btposmin': 8, 'btposnum': 3, 
                     'btnegavg': 2, 'btnegmax': 8, 'btnegmin': 8, 'btnegnum': 3, 
                     'cn': 1, 'gcn': 0, 'scn': 3, 'sgcn': 3, 'q6q6': 2, 
                     'Ixx': 5, 'Iyy': 5, 'Izz': 5, 'degenDeg': 6, 
                     'angParam': 4, 'centParam': 4, 'entroParam': 5, 'entroAvgParam': 5.5, 
                     'chi1': 6, 'chi2': 6, 'chi3': 6, 'chi4': 6, 'chi5': 6, 'chi6': 6, 'chi7': 6, 'chi8': 6, 'chi9': 6, 
                     'q2': 5.7, 'q4': 5.6, 'q6': 5.5, 'q8': 5.7, 'q10': 5.7, 'q12': 5.7, 
                     'q2avg': 5.2, 'q4avg': 5.1, 'q6avg': 5, 'q8avg': 5.2, 'q10avg': 5.2, 'q12avg': 5.2, 
                     'disord2': 4.7, 'disord4': 4.6, 'disord6': 4.5, 'disord8': 4.7, 'disord10': 4.7, 'disord12': 4.7, 
                     'disordAvg2': 4.2, 'disordAvg4': 4.1, 'disordAvg6': 4, 'disordAvg8': 4.2, 'disordAvg10': 4.2, 'disordAvg12': 4.2
                     }  # Lower score = Higher rank

    # occurCount = Counter(list(sum(highCorCols, ())))
    print("\n    Sorting all highly-correlated feature pairs based on their minimum and total utility rankings.")
    highCorColsProps = []
    for (col1, col2) in highCorCols:
        rank1, rank2 = utilityRanking[col1], utilityRanking[col2]
        highCorColsProps.append((min(rank1, rank2), rank1 + rank2))
    sortedIdx = sorted(range(len(highCorColsProps)), key=lambda i: highCorColsProps[i])
    highCorCols, highCorColsProps = [highCorCols[i] for i in sortedIdx], [highCorColsProps[i] for i in sortedIdx]
    
    print("\n    Removing one of each highly-correlated feature pairs.")
    cols2drop = []
    for (i, (col1, col2)) in enumerate(highCorCols):
        if verbose: 
            print(f"      Feature pairs: {col1} {col2}")
        if col1 in cols2drop or col2 in cols2drop:
            if verbose: 
                print("        One of the features is dropped, skip this pair.\n")
            continue
        elif utilityRanking[col1] > utilityRanking[col2]:
            if verbose: 
                print(f"        {col1} has lower utility score compared to {col2}")
            cols2drop.append(col1)
        else:
            if verbose: 
                print(f"        {col2} has lower utility score compared to {col1}")
            cols2drop.append(col2)
    if verbose: 
        print(f"    Feature columns to drop: {cols2drop}")
    return cols2drop

In [15]:
def varCorrDropCols(X, varThresh=0.00, corrThresh=0.90, figName=None, verbose=False):
    """
    Remove features with low variance and one of the highly-correlated feature pairs
    input:
        X = input scaled DataFrame with each column being feature
        varThresh = threshold below which feature is removed
        corrThresh = threshold above which one of each pair of correlated features is removed
        verbose = Boolean indicator for output printing
    output:
        XNoLVHC = DataFrame with the undesired features removed
        corrMatNoLVHC = computed correlated matrix
    """
    # Remove columns with low variance
    XNoLV, lowVarCols = getLowVarCols(df=X, skipCols=None, varThresh=varThresh, autoRemove=True)
    
    # Remove one of the feature columns that are highly correlated with each other
    corrMatNoLV, highCorCols1 = getHighCorCols(df=XNoLV, corrThresh=corrThresh, method=CORR_METHOD)
    
    cols2drop = autoDrop(highCorCols1, verbose=verbose)
    XNoLVHC = XNoLV.drop(labels=cols2drop, axis=1, index=None, columns=None, level=None, inplace=False, errors="raise")
    corrMatNoLVHC, highCorCols2 = getHighCorCols(df=XNoLVHC, corrThresh=corrThresh, method=CORR_METHOD)
    plotCorrMat(corrMat=corrMatNoLVHC, figSize=(8, 8), figName=figName)
    return XNoLVHC, corrMatNoLVHC

<a id='data-vis'></a>
### Data Visualisation

##### t-SNE
- perplexity = related to number of nearest neighbours used in other manifold learning algorithms, larger datasets need larger value, range = (5-50), different values give significantly different results
- early_exaggeration = controls how tight natural clusters in original space are in embedded space, proportional to space between clusters in embedded space
- learning_rate = range = \[10.0, 1000.0\], too high = ball with equidistant points, too low = most points look compressed in a dense cloud with some outliers

In [16]:
def vis2D(caseID, varThreshIdx=0, corrThreshIdx=1, colourLabel='surf',
          perplexity=30.0, early_exaggeration=12, metric='euclidean'):
    """
    Visualise the atoms in 2D manifold using PCA, t-SNE and UMAP
    input:
        caseID = ID index of nanoparticle of interest
        varThresh = threshold below which feature is removed
        corrThresh = threshold above which one of each pair of correlated features is removed
        * Refer to sklearn and umap packages for other input variables
    """
    %store -r dfScaledNoLVHCs
    df = dfScaledNoLVHCs[varThreshIdx][corrThreshIdx][caseID]
    X, y = df.iloc[:, :-1], df[colourLabel]
    tsne = TSNE(n_components=2, 
                perplexity=perplexity, early_exaggeration=early_exaggeration, learning_rate='auto', 
                n_iter=1000, n_iter_without_progress=300, min_grad_norm=1e-7, metric=metric, 
                random_state=RANDOM_SEED, init='pca', verbose=0, 
                method='barnes_hut', angle=0.5, n_jobs=NUM_JOBS)
    redTSNE = tsne.fit_transform(X)

    sns.set_style('ticks')
    fig, ax = plt.subplots(figsize=(3, 3))
    colours = [BINARY_COLOURS[0] if isSurf else BINARY_COLOURS[1] for isSurf in y] if colourLabel == 'surf' else y
    plt.scatter(redTSNE[:, 0], redTSNE[:, 1], s=3, c=colours, cmap=plt.get_cmap('viridis', len(y.unique())))
    plt.xlabel('Dimension 1')
    plt.ylabel('Dimension 2')
    plt.grid(linestyle='dotted')
    plt.savefig(f"figures/dimRedVis/{testCases[caseID]}DRvis_V{int(varThreshs[varThreshIdx]*100)}C{int(corrThreshs[corrThreshIdx]*100)}{colourLabel}.png",
                dpi=DPI, bbox_inches='tight')
    # plt.close()

In [17]:
def expVarPCA(caseID, varThreshIdx=0, corrThreshIdx=1, verbose=False):
    """
    Calculate and plot the variance explained by PCA components
    input:
        caseID = ID index of nanoparticle of interest
        varThresh = threshold below which feature is removed
        corrThresh = threshold above which one of each pair of correlated features is removed
        * Refer to sklearn package for other input variables
    """
    %store -r dfScaledNoLVHCs
    pca = PCA(n_components=PCA_EXP_VAR_THRESHOLD, copy=True, whiten=False, svd_solver='auto', tol=0.0, 
              iterated_power='auto', random_state=RANDOM_SEED)
    pca.fit_transform(dfScaledNoLVHCs[varThreshIdx][corrThreshIdx][caseID].iloc[:, :-1])
    
    expVarPCA = pca.explained_variance_ratio_
    cumSumEigenvals = np.cumsum(expVarPCA)
    sns.set_style('ticks')
    fig, ax = plt.subplots(figsize=(3, 2.5))
    plt.bar(range(0, len(expVarPCA)), expVarPCA, color=BINARY_COLOURS[0], alpha=0.5, align='center', label='Individual')
    plt.step(range(0, len(cumSumEigenvals)), cumSumEigenvals, where='mid', color=BINARY_COLOURS[0], label='Cumulative')
    # plt.figure(figsize=(4, 3.5))
    # plt.plot(np.cumsum(pca.explained_variance_ratio_), marker=None, color='k')
    plt.xlabel('Principal component index')
    plt.ylabel('Explained variance ratio')
    plt.grid(linestyle='dotted')
    plt.savefig(f"figures/PCAexpVar/{testCases[caseID]}PCA_V{int(varThreshs[varThreshIdx]*100)}C{int(corrThreshs[corrThreshIdx]*100)}.png",
                dpi=DPI, bbox_inches='tight')
    # plt.close()
    if verbose:
        # print("PCA components:\n", pca.components_)
        # print("Explained Variance:\n", pca.explained_variance_)
        print("\nExplained Variance Ratio:\n", expVarPCA)

In [18]:
def dimRedVis(caseID, varThreshIdxs=[0], corrThreshIdxs=[1], colourLabel='surf', verbose=False):
    """
    Visualise data with reduced dimensions. 
    input:
        caseID = ID index of nanoparticle of interest
        varThresh = threshold below which feature is removed
        corrThresh = threshold above which one of each pair of correlated features is removed
        * Refer to sklearn package for other input variables
    output:
    """
    %store -r dfScaledNoLVHCs dfNoLVHCs
    for varThreshIdx in varThreshIdxs:
        for corrThreshIdx in corrThreshIdxs:
            vis2D(caseID=caseID, varThreshIdx=varThreshIdx, corrThreshIdx=corrThreshIdx, colourLabel=colourLabel)
            expVarPCA(caseID=caseID, varThreshIdx=varThreshIdx, corrThreshIdx=corrThreshIdx, verbose=verbose)
            if verbose:
                print("\nScaled DataFrame, varThresh: {0}, corrThresh {1}:".format(varThreshs[varThreshIdx], corrThreshs[corrThreshIdx]))
                # display(dfScaledNoLVHCs[varThreshIdx][corrThreshIdx][caseID].sample(5))
                print("\nUnscaled DataFrame, varThresh: {0}, corrThresh {1}:".format(varThreshs[varThreshIdx], corrThreshs[corrThreshIdx]))
                # display(dfNoLVHCs[varThreshIdx][corrThreshIdx][caseID].sample(5))
                
# dimRedVis(caseID=i, varThreshIdxs=[0], corrThreshIdxs=[0], colourLabel='surf', verbose=True)

<a id='complete-pipeline'></a>
### Complete Pipeline

In [19]:
def dataPreProc(caseID, verbose=False, interactive=True):
    """
    Run the data preprocessing pipeline for a nanoparticle
    input:
        caseID = ID index of nanoparticle of interest
        verbose = Boolean indicator of whether text outputs should be elaborated 
        interactive = Boolean indicator of whether DataFrames should be displayed (to be investigated in Notebook) or printed (to be stored in .txt file)
    output:
        dfNoLVHC = unscaled DataFrame without highly-correlated and low variance features
        dfScaledNoLVHC = scaled DataFrame without highly-correlated and low variance features
        corrMatNoLVHC = computed correlation matrix
    """
    print("\nTest case: {0}".format(testCases[caseID]))
    # dfDesc(df=dfAlls[caseID], interactive=interactive)
    dfCleaned = dfMassage(df=dfAlls[caseID], verbose=verbose)
    dfDropped = dfDropUnusedCols(df=dfCleaned)
    dfScaled = dfScale(df=dfDropped, catCols=['surf'], scaler='minMax')
    Xscaled, y = dfScaled.iloc[:, :-1], dfScaled.loc[:, 'surf']
    
    %store -r dfScaledNoLVHCs dfNoLVHCs corrMatNoLVHCs
    for (i, varThresh) in enumerate(varThreshs):
        for (j, corrThresh) in enumerate(corrThreshs):
            print(f"\nRemove features with varThresh: {varThresh}, corrThresh: {corrThresh}")
            varCorrDropColsFigName = f"figures/corrMat/{testCases[caseID]}cmatV{int(varThresh*100)}C{int(corrThresh*100)}.png"
            XScaledNoLVHC, corrMatNoLVHC = varCorrDropCols(X=Xscaled, varThresh=varThresh, corrThresh=corrThresh, figName=varCorrDropColsFigName, verbose=verbose)
            dfScaledNoLVHC = pd.concat(objs=[XScaledNoLVHC, y], axis=1, join="inner", ignore_index=False)
            dfNoLVHC = dfDropped[dfScaledNoLVHC.columns]
            dfScaledNoLVHCs[i][j][caseID] = dfScaledNoLVHC
            dfNoLVHCs[i][j][caseID] = dfNoLVHC
            corrMatNoLVHCs[i][j][caseID] = corrMatNoLVHC
    %store dfScaledNoLVHCs dfNoLVHCs corrMatNoLVHCs

# dataPreProc(0, verbose=False, interactive=False)

<a id='test-cases'></a>
## Test Cases

In [29]:
# i=0

In [144]:
%%capture dataProcPd --no-stderr
if i < len(testCases): 
    dataPreProc(caseID=i, verbose=True)

In [145]:
%%capture dimRedVisPd --no-stderr
if i < len(testCases): 
    dimRedVis(caseID=i, varThreshIdxs=[0], corrThreshIdxs=[0], colourLabel='surf', verbose=True)

In [146]:
print(f"Case {i}: {testCases[i]}") if i < len(testCases) else print('No more test cases!')
with open(f"dataPreprocOutputs/dataProc{i}Pd.txt", 'w') as f: 
    f.write(dataProcPd.stdout)
with open(f"dataPreprocOutputs/dimRedVis{i}Pd.txt", 'w') as f: 
    f.write(dimRedVisPd.stdout)
i += 1

No more test cases!


<a id='var-store'></a>
## Variables Storage

In [147]:
with open('data/dfScaledNoLVHCs.pickle', 'wb') as f: 
    pickle.dump(dfScaledNoLVHCs, f)
with open('data/dfNoLVHCs.pickle', 'wb') as f: 
    pickle.dump(dfNoLVHCs, f)
with open('data/corrMatNoLVHCs.pickle', 'wb') as f: 
    pickle.dump(corrMatNoLVHCs, f)

In [148]:
# Check the output
with open("data/dfScaledNoLVHCs.pickle", 'rb') as f: 
    dfScaledNoLVHCs = pickle.load(f)
for (i, dfScaledNoLVHC) in enumerate(dfScaledNoLVHCs[0][0]):  # varThresh = 0.0, corrThresh = 0.9
    display(dfScaledNoLVHC.sample(5))

,x,y,z,ba1max,ba2avg,btposavg,gcn,scn,rad,q2avg,q4avg,q6avg,surf
347,0.333333,0.750000,0.500000,1.0,0.269231,0.656934,1.000000,0.000000,0.382948,0.000066,2.499172e-08,7.927517e-08,0
34,0.750000,0.666667,0.000000,1.0,0.000000,1.000000,0.141026,0.833333,0.881503,0.762871,5.936802e-01,4.600804e-01,1
268,0.833333,0.833333,0.250000,1.0,0.096154,0.766423,0.423077,1.000000,0.793353,1.000000,4.699697e-01,9.999999e-01,1
629,0.083333,0.666667,0.833333,1.0,0.346154,0.613139,0.384615,1.000000,0.838150,0.863204,3.895629e-01,7.327783e-01,1
535,0.416667,0.833333,0.666667,1.0,0.269231,0.510949,1.000000,0.000000,0.526012,0.000118,2.205624e-08,8.704122e-08,0


,x,y,z,blavg,blmax,ba1avg,ba1max,ba1min,ba2avg,btposavg,btnegavg,gcn,rad,q6q6,chi3,q2,q4,q6,q10,q2avg,q6avg,centParam,surf
503,0.718171,0.686341,0.230326,0.606557,1.0,1.000000,1.0,0.0,0.582090,0.103960,0.896040,0.896104,0.669977,1.000000,0.0,0.141245,0.943637,0.596507,0.076420,0.444519,0.827572,4.905885e-09,0
789,0.127318,0.833333,0.500000,0.803279,1.0,0.868996,1.0,0.0,0.492537,0.143564,0.856436,0.376623,0.850467,0.750000,0.0,0.986282,0.999951,0.687487,0.245562,0.873621,0.645029,4.362793e-01,1
596,0.872682,0.166667,0.603008,0.901639,1.0,0.672489,1.0,0.0,0.119403,0.608911,0.391089,0.259740,0.868575,0.666667,0.2,0.999835,0.597099,0.457686,0.443945,0.955075,0.659531,4.024708e-01,1
258,0.718171,0.365163,0.686341,0.606557,1.0,1.000000,1.0,0.0,0.402985,0.193069,0.806931,1.000000,0.539136,1.000000,0.0,0.141254,0.943640,0.596504,0.076438,0.164442,0.687355,6.307566e-09,0
525,0.250000,0.500000,0.821178,0.606557,1.0,1.000000,1.0,0.0,0.611940,0.000000,1.000000,0.896104,0.692173,1.000000,0.0,0.141305,0.943658,0.596488,0.076530,0.429752,0.682557,3.504203e-09,0


,x,y,z,ba1avg,ba1max,ba2avg,btposavg,gcn,scn,rad,entroAvgParam,q4,q4avg,q6avg,surf
108,0.357143,0.285714,0.285714,1.000000,1.0,0.596491,0.328571,0.897436,0.0,0.615633,0.182256,1.657348e-08,7.516610e-02,1.095470e-01,0
374,0.071429,0.571429,0.571429,1.000000,1.0,0.824561,0.128571,0.769231,0.0,0.809432,0.420557,1.657348e-08,3.263967e-01,6.408548e-02,0
237,0.357143,0.071429,0.357143,0.802469,1.0,0.403509,0.571429,0.384615,1.0,0.870801,0.653699,3.250711e-01,5.549321e-01,7.327875e-01,1
707,0.428571,0.571429,0.785714,1.000000,1.0,0.333333,0.550000,1.000000,0.0,0.556848,0.000009,4.972321e-09,3.141713e-08,8.704121e-08,0
218,0.214286,0.357143,0.357143,1.000000,1.0,0.596491,0.328571,0.897436,0.0,0.643411,0.182255,4.972321e-09,7.516609e-02,1.095470e-01,0


,x,y,z,ba1avg,ba1max,ba2avg,btposavg,btnegavg,gcn,scn,rad,entroParam,entroAvgParam,q4,q6,q4avg,q6avg,surf
391,0.000000,0.083333,0.500000,0.000000,0.0,0.000000,1.000000,0.000000,0.051948,0.833333,0.875395,0.999987,0.812995,4.143433e-08,9.757974e-08,4.304578e-01,6.054088e-01,1
1002,0.750000,0.666667,1.000000,0.623457,1.0,0.017241,0.800000,0.200000,0.311688,0.666667,0.771664,0.679645,0.579645,9.999195e-01,3.576895e-02,9.999052e-01,2.917230e-01,1
756,0.083333,1.000000,0.833333,0.000000,0.0,0.137931,0.819355,0.161290,0.000000,0.833333,1.000000,0.999996,0.957842,4.143433e-08,9.757974e-08,1.411261e-01,9.999912e-01,1
525,0.833333,0.750000,0.500000,1.000000,1.0,0.172414,0.470968,0.529032,1.000000,0.000000,0.514231,0.000009,0.000005,4.972120e-09,4.440937e-08,1.227218e-08,5.572134e-07,0
138,0.500000,0.666667,0.083333,1.000000,1.0,0.517241,0.348387,0.651613,0.831169,0.000000,0.564200,0.000009,0.321034,4.972120e-09,4.440935e-08,3.228164e-01,8.926434e-02,0


,x,y,z,ba1avg,ba1max,ba2avg,btposavg,btnegavg,gcn,scn,rad,entroAvgParam,q6,q4avg,q6avg,surf
1001,0.500000,0.333333,0.916667,1.00000,1.0,0.439024,0.435556,0.564444,0.852273,0.0,0.495831,0.260377,4.034982e-08,0.322855,0.036581,0
598,0.000000,0.666667,0.583333,0.69802,1.0,0.085366,0.768889,0.231111,0.397727,0.8,0.610339,0.470125,3.239068e-02,0.999727,0.118894,1
1077,0.916667,0.166667,1.000000,0.69802,1.0,0.365854,0.528889,0.471111,0.295455,1.0,0.878822,0.695178,3.252635e-02,0.702920,0.662639,1
2,0.000000,0.416667,0.000000,0.00000,0.0,0.000000,1.000000,0.000000,0.056818,0.8,0.852696,0.803444,9.998662e-01,0.747025,0.413964,1
88,0.000000,0.250000,0.166667,0.69802,1.0,0.085366,0.688889,0.311111,0.397727,0.8,0.769316,0.470125,3.239068e-02,0.999727,0.118894,1


...

,x,y,z,blavg,blmax,ba1avg,ba1max,ba1min,ba2avg,ba2max,ba2min,btposavg,btnegavg,btnegmax,btnegmin,btposmax,btposmin,gcn,scn,rad,entroAvgParam,chi2,chi3,chi4,chi7,q4,q6,q8,q10,q12,q2avg,q6avg,q10avg,q12avg,centParam,surf
105,0.585000,0.569970,0.137840,0.583851,0.843318,0.836820,0.967742,0.529412,0.239130,0.272727,0.000000,0.564815,0.362385,1.0,0.0,1.0,0.0,0.415730,0.750,0.481106,0.477526,0.0,0.0,0.0,0.0,0.528403,0.755202,0.731034,0.425008,0.838038,0.601924,0.867135,0.525333,0.746536,0.440125,1
140,0.892224,0.348288,0.221918,0.701863,0.748080,0.472803,0.790323,0.352941,0.206522,0.272727,0.111111,0.680556,0.266055,1.0,0.0,1.0,0.0,0.224719,0.625,0.738710,0.666204,0.0,0.0,0.0,0.0,0.591090,0.541136,0.667188,0.390802,0.547073,0.339940,0.614731,0.199771,0.592534,0.637162,1
946,0.744197,0.172528,0.822072,0.509317,0.305684,0.569038,0.870968,0.470588,0.402174,0.363636,0.000000,0.587963,0.311927,0.5,0.0,1.0,0.5,0.213483,0.625,0.764516,0.737967,0.0,0.0,0.0,0.0,0.594440,0.668543,0.628922,0.577625,0.670512,0.395183,0.466542,0.547634,0.563190,0.645710,1
219,0.516543,0.401651,0.257637,0.586957,0.614439,0.979079,0.967742,0.529412,0.500000,0.318182,0.000000,0.412037,0.449541,1.0,0.0,1.0,0.0,0.898876,0.000,0.323502,0.180853,0.0,0.0,0.0,0.0,0.406794,0.594639,0.584543,0.218022,0.690880,0.203675,0.741467,0.129902,0.673647,0.023535,0
463,0.403464,0.805828,0.562633,0.624224,0.536098,0.828452,0.951613,0.352941,0.228261,0.272727,0.111111,0.638889,0.302752,1.0,0.0,1.0,0.0,0.483146,0.750,0.497696,0.452319,0.0,0.0,0.0,0.0,0.557982,0.652508,0.722613,0.380902,0.737407,0.546300,0.833887,0.272996,0.761236,0.440504,1


,x,y,z,blavg,blmax,ba1avg,ba1max,ba1min,ba2avg,ba2max,ba2min,btposavg,btnegavg,btnegmax,btnegmin,btposmax,btposmin,gcn,scn,rad,q6q6,entroAvgParam,chi2,chi3,chi4,chi7,q4,q6,q8,q10,q12,q2avg,q4avg,q6avg,q10avg,q12avg,centParam,degenDeg,surf
837,0.400395,0.413469,0.582533,0.762040,0.609262,0.980296,0.918033,0.500000,0.298077,0.20,0.0,0.616,0.497717,1.000000,0.00,1.000000,0.000000,0.976744,0.00,0.192780,0.857143,0.020547,0.0,0.0,0.0,0.000000,0.342295,0.751471,0.608616,0.083736,0.755113,0.057727,0.674965,0.901391,0.108912,0.904336,0.018245,1.0,0
640,0.462015,0.387388,0.558860,0.682720,0.539797,1.000000,0.983607,0.555556,0.298077,0.20,0.0,0.676,0.465753,1.000000,0.00,1.000000,0.000000,0.976744,0.00,0.164124,0.857143,0.025669,0.0,0.0,0.0,0.000000,0.367768,0.782774,0.692874,0.167344,0.879499,0.076957,0.678621,0.902519,0.138474,0.910638,0.002172,1.0,0
1031,0.345483,0.068313,0.719028,0.614731,0.499276,0.990148,0.967213,0.388889,0.711538,0.25,0.1,0.328,0.771689,1.000000,0.00,1.000000,0.000000,0.709302,0.00,0.724228,0.857143,0.467915,0.0,0.0,0.0,0.000000,0.347421,0.766157,0.613270,0.239948,0.763014,0.237462,0.681776,0.834724,0.339738,0.674214,0.008987,1.0,0
69,0.130642,0.330667,0.061869,0.654391,0.518090,0.660099,0.803279,0.277778,0.201923,0.55,0.1,0.556,0.465753,1.000000,0.00,1.000000,0.000000,0.372093,0.75,0.837365,0.642857,0.597748,0.0,0.0,0.0,0.000000,0.369368,0.702159,0.451534,0.328737,0.387309,0.541688,0.448233,0.657811,0.382174,0.458954,0.425371,1.0,1
1293,0.430671,0.065712,0.946884,0.810198,0.931983,0.310345,0.754098,0.222222,0.326923,0.70,0.0,0.636,0.456621,0.333333,0.25,0.666667,0.333333,0.186047,0.75,0.905471,0.571429,0.824332,0.0,0.0,0.0,0.666667,0.268567,0.699152,0.651543,0.500775,0.355523,0.276586,0.206882,0.610233,0.650393,0.208635,0.631884,1.0,1


,x,y,z,blavg,blmax,ba1avg,ba1max,ba1min,ba2avg,ba2max,ba2min,btposavg,btnegavg,btnegmax,btnegmin,btposmax,btposmin,gcn,scn,rad,q6q6,entroParam,entroAvgParam,chi2,chi3,chi4,chi6,chi7,q2,q4,q6,q8,q10,q12,q2avg,q4avg,q6avg,q10avg,q12avg,centParam,degenDeg,surf
640,0.755900,0.712438,0.805982,0.804071,0.904903,0.711111,0.955882,0.000000,0.298851,0.391304,0.083333,0.521739,0.525316,1.0,0.0,1.0,0.00,0.372340,0.857143,0.706096,0.357143,0.596808,0.595358,0.333333,0.000000,0.00,0.333333,0.25,0.552847,0.407210,0.721596,0.600575,0.397748,0.595391,0.815288,0.314922,0.736369,0.443680,0.505485,0.492566,1.0,1
586,0.471242,0.201712,0.827824,0.610687,0.684993,0.924444,0.867647,0.470588,0.494253,0.521739,0.083333,0.478261,0.550633,1.0,0.0,1.0,0.00,0.457447,0.857143,0.692308,0.285714,0.449940,0.530344,0.666667,0.333333,0.25,0.333333,0.25,0.646019,0.098104,0.506120,0.072470,0.410826,0.484268,0.729596,0.000861,0.573115,0.348719,0.372827,0.217179,1.0,1
437,0.289337,0.792225,0.673385,0.740458,0.710253,0.924444,0.955882,0.294118,0.321839,0.608696,0.000000,0.496894,0.601266,0.5,0.0,1.0,0.25,0.893617,0.000000,0.648766,0.714286,0.087424,0.181576,0.000000,0.222222,0.25,0.600000,0.50,0.155466,0.173512,0.439714,0.478807,0.380137,0.184995,0.057786,0.368965,0.607559,0.224045,0.236449,0.076943,1.0,0
446,0.282297,0.173312,0.527017,0.600509,0.898960,0.946667,0.970588,0.294118,0.620690,0.695652,0.000000,0.279503,0.936709,1.0,0.0,1.0,0.00,0.680851,0.000000,0.656749,0.428571,0.307339,0.434270,0.333333,0.333333,0.50,0.400000,0.75,0.299861,0.411383,0.454788,0.153395,0.531588,0.320807,0.259438,0.826813,0.646284,0.391834,0.292500,0.194244,1.0,0
505,0.590873,0.724604,0.665952,0.679389,0.738484,0.960000,0.955882,0.470588,0.321839,0.478261,0.083333,0.403727,0.594937,1.0,0.0,1.0,0.00,0.851064,0.000000,0.449202,0.857143,0.229229,0.091323,0.000000,0.000000,0.25,0.666667,0.00,0.140654,0.386907,0.574793,0.516144,0.263345,0.443181,0.207221,0.532463,0.782634,0.288479,0.514407,0.068242,1.0,0


,x,y,z,blavg,blmax,ba1avg,ba1max,ba1min,ba2avg,ba2max,ba2min,btposavg,btnegavg,btnegmax,btnegmin,btposmax,btposmin,gcn,scn,rad,q6q6,entroParam,entroAvgParam,chi2,chi3,chi4,chi6,chi7,q2,q4,q6,q8,q10,q12,q2avg,q4avg,q6avg,q8avg,q10avg,q12avg,centParam,degenDeg,surf
703,0.800052,0.627439,0.521000,0.624260,0.676573,0.902542,0.793651,0.526316,0.454545,0.363636,0.000000,0.213018,0.588235,1.000000,0.0,1.0,0.000000,0.683673,0.000000,0.574984,0.846154,0.246978,0.309619,0.666667,0.333333,0.2,0.733333,0.333333,0.302477,0.133241,0.441828,0.376981,0.214773,0.195099,0.286088,0.482074,0.746985,0.571480,0.499574,0.360593,0.226007,1.0,0
466,0.470144,0.340862,0.602331,0.650888,0.568182,0.881356,0.857143,0.473684,0.353535,0.500000,0.285714,0.497041,0.605882,1.000000,0.0,1.0,0.000000,0.867347,0.000000,0.288147,0.692308,0.101862,0.171012,0.333333,0.111111,0.4,0.600000,0.666667,0.142681,0.069887,0.531899,0.495844,0.246078,0.214949,0.058328,0.417149,0.644750,0.403387,0.299829,0.311025,0.138281,1.0,0
209,0.175801,0.645383,0.411716,0.600592,0.804196,0.906780,0.952381,0.368421,0.424242,0.272727,0.285714,0.355030,0.664706,1.000000,0.0,1.0,0.000000,0.785714,0.000000,0.650950,0.923077,0.345134,0.085574,0.333333,0.000000,0.0,0.600000,0.000000,0.166727,0.347532,0.706918,0.516371,0.371621,0.499346,0.047282,0.578253,0.785120,0.609161,0.168692,0.415312,0.090611,1.0,0
415,0.658836,0.669297,0.233274,0.508876,0.639860,0.885593,0.904762,0.263158,0.737374,0.909091,0.000000,0.201183,0.823529,1.000000,0.0,1.0,0.000000,0.693878,0.000000,0.602489,0.923077,0.375270,0.396915,0.000000,0.000000,0.4,0.533333,0.333333,0.241354,0.400823,0.569729,0.454733,0.364719,0.253351,0.184042,0.665136,0.817089,0.643458,0.693010,0.438481,0.239183,1.0,0
33,0.167850,0.308082,0.220754,0.349112,0.886364,0.889831,0.904762,0.526316,0.656566,0.500000,0.428571,0.337278,0.623529,0.833333,0.0,1.0,0.111111,0.183673,0.857143,0.848723,0.307692,0.780186,0.758814,0.000000,0.000000,0.0,0.066667,0.000000,0.765731,0.707723,0.879384,0.448918,0.876428,0.458334,0.552994,0.478091,0.688183,0.407515,0.519637,0.469946,0.211231,1.0,1


,x,y,z,blavg,blmax,ba1avg,ba1max,ba1min,ba2avg,ba2max,ba2min,btposavg,btnegavg,btnegmax,btnegmin,btposmax,btposmin,gcn,scn,rad,q6q6,entroParam,entroAvgParam,chi2,chi3,chi4,chi5,chi6,chi7,q2,q4,q6,q8,q10,q12,q2avg,q4avg,q6avg,q10avg,q12avg,centParam,degenDeg,surf
688,0.671727,0.412960,0.605858,0.636086,0.537786,0.943723,0.964286,0.5000,0.512605,0.423077,0.1,0.50000,0.556213,1.0,0.0,1.000000,0.0,0.791209,0.0,0.364012,0.571429,0.193513,0.073011,0.5,0.3,0.25,0.689655,0.6875,0.00,0.066369,0.111919,0.531558,0.444290,0.117247,0.623122,0.180901,0.141024,0.480497,0.038872,0.749159,0.182255,1.0,0
330,0.519943,0.417096,0.401426,0.498471,0.448155,0.974026,0.982143,0.6875,0.521008,0.269231,0.1,0.44375,0.562130,1.0,0.0,1.000000,0.0,0.868132,0.0,0.197050,0.857143,0.210199,0.117335,0.0,0.0,0.00,0.758621,0.7500,0.00,0.056962,0.286597,0.776726,0.656153,0.044300,0.817572,0.059822,0.503374,0.641860,0.254392,0.778782,0.020754,1.0,0
870,0.488040,0.843895,0.597324,0.452599,0.766257,0.956710,0.982143,0.5625,0.512605,0.500000,0.2,0.37500,0.710059,1.0,0.5,0.666667,0.0,0.780220,0.0,0.614749,0.857143,0.295235,0.152943,0.0,0.0,0.00,0.620690,0.6250,0.00,0.221062,0.340949,0.722552,0.619841,0.106157,0.691037,0.009699,0.616305,0.666118,0.269076,0.610098,0.037554,1.0,0
383,0.622568,0.526564,0.330529,0.553517,0.697715,0.965368,0.964286,0.6250,0.495798,0.384615,0.0,0.51250,0.526627,1.0,0.0,1.000000,0.0,0.725275,0.0,0.352802,0.571429,0.206058,0.125678,0.0,0.0,0.00,0.793103,0.6250,0.25,0.091754,0.256281,0.726757,0.563543,0.158911,0.626700,0.107782,0.540691,0.515254,0.138350,0.330482,0.042770,1.0,0
491,0.656577,0.257556,0.624451,0.550459,0.738137,0.965368,0.946429,0.4375,0.663866,0.538462,0.0,0.34375,0.715976,1.0,0.0,1.000000,0.0,0.714286,0.0,0.538053,0.857143,0.193789,0.291600,0.0,0.0,0.00,0.758621,0.6875,0.00,0.123607,0.264372,0.760703,0.599156,0.128980,0.715463,0.246043,0.545235,0.688482,0.397480,0.704159,0.027354,1.0,0
